<a href="https://colab.research.google.com/github/pangxiangyang/StockPricePrediction/blob/master/Data_extraction_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
inipath = "/content/gdrive/My Drive/NewData"

files = []
directories = []
# r=root, d=directories, f = files
for r, d, f in os.walk(inipath):
    if( r.find(".MY") > -1 ):
      directories.append(r)

total = 0
for folder in directories:
  source_files= folder + '/*.zip';
  filelist=glob.glob(source_files)
  #print( folder, len( filelist) )
  if(len(filelist)==28 ):
    print(folder)
  total = total+len(filelist);

print( total)

6570


In [0]:
# Run following scripts to install chrome driver in new environment
#!apt-get update # to update ubuntu to correctly run apt install
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [0]:
def SeleniumSetup():
  # Setup google selenium for web scraping
  import sys
  sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [0]:
def ChromeSetup( isMobile = False ):
  from selenium import webdriver
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  if( isMobile ):
    chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
  driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  return driver

In [0]:
def GoogleAuthentication():
  # Google account authentication
  from google.colab import auth
  auth.authenticate_user()

  import gspread
  from oauth2client.client import GoogleCredentials

  from google.colab import drive
  drive.mount('/content/gdrive')

  gc = gspread.authorize(GoogleCredentials.get_application_default())
  return gc;

In [0]:
def GetGoogleSpreadSheet(gc):
  # Get started with list of company
  url = ""
  spreadsheet = gc.open_by_url(url)
  return spreadsheet

In [0]:
def GetCompanyList( spreadsheet ):
  worksheet = spreadsheet.sheet1;
  # get codes 
  numerical_codes = worksheet.range('B2:B950');
  names           = worksheet.range('A2:A950');
  isimportrequired = worksheet.range('C2:C950')

  codes = []
  for i in range( len( numerical_codes ) ):
    if( isimportrequired[i].value == "#N/A" ):
      codes.append( numerical_codes[i].value.strip() + ".MY" )
  return codes;

In [0]:
# Login into shareinvestors.com website
def Login( driver ):
  driver.get('http://www.shareinvestor.com/user/login.html');
  driver.get_screenshot_as_file("fff.png")
  username_box = driver.find_element_by_id('sic_login_header_username')
  username_box.send_keys('')

  password_box = driver.find_element_by_id('sic_login_header_password')
  password_box.send_keys('')
  login_btn = driver.find_element_by_id('sic_login_submit')
  login_btn.submit();
  return driver;

In [0]:
def GetDateList( spreadsheet ):
  # Get date
  worksheet = spreadsheet.get_worksheet(1)
  dates = worksheet.col_values(1)
  return dates;

In [0]:
class DownloadAction:
  def __init__(self, name, istest, isremovefile,url,sourcefile, targetfile, 
               codes, driver, date =[], companyrange=0, daterange=0, isadjust = False,
               adjustmentlist ={} ):
    self.url = url
    self.Name         = name
    self.CompanyRange = companyrange
    self.DateRange    = daterange
    self.IsTest       = istest
    self.IsRemoveFile = isremovefile
    self.SourceFile   = sourcefile;
    self.TargetFile   = targetfile;
    self.CompanyList  = codes;
    self.Dates        = date;
    self.Driver       = driver;
    self.Codes        = codes;
    self.IsAdjust     = isadjust
    self.List         = adjustmentlist

  def Download( self, driver ):
      # Download historical data
      try:
        count = 1;
        actualRange = self.GetCompanyRange()
        codes = self.Codes
        initialpath = self.TargetFile
        for code in codes[0:actualRange]:
          if( len( self.Dates ) == 0):#Normal download
            action_url = self.url.replace("XXXX", code)
            print( count, action_url )
            self.Execute(action_url);
          else: # Daily transactional data download
            # Create directory to store downloaded file
            filepath = initialpath + code + "/"
            if( not path.exists(filepath) ):
              os.mkdir( filepath )
            self.TargetFile = filepath;
            print( count, filepath )
            dates = self.Dates
            timecount = 1;
            actualDateRange = self.DateRange
            if( self.DateRange == 0 ):
              actualDateRange = len(self.Dates)
            for tradedate in dates[0:actualDateRange]:
              action_url = self.url.replace("XXXX", code)
              action_url = action_url.replace("yyyyyyyy", tradedate );
              print(timecount, action_url)
              dictionary = self.List;
              try:
                if( self.IsAdjust ):
                  requiredDates = dictionary[code];
                  requiredDates.index(tradedate);
                  self.Execute(action_url );
                else:
                  self.Execute(action_url);
              except Exception as e:
                print ( code, tradedate );
                #self.Execute(action_url );
              timecount = timecount + 1;
          count = count + 1;
      except Exception as e:
        print( e )
        self.MoveFiles();
        SendEmail( str(e) )
  
  def MoveFiles( self ):# Copy source files to target files. 
      source_files = self.SourceFile
      filepath = self.TargetFile
      filelist=glob.glob(source_files)
      for single_file in filelist:
      # move file with full paths as shutil.move() parameters
        print( single_file, filepath)
        shutil.copy(single_file, filepath)
        if(self.IsRemoveFile == True ):
          os.remove(single_file)
          
  def Randomize( self):
    import random
    return random.randint(15,20)
    
  def GetCompanyRange(self):
    actualRange = self.CompanyRange;
    codes = self.Codes
    print(len(codes), actualRange)
    if( self.CompanyRange == 0 ):
      actualRange = len(codes)
    return actualRange

  def Execute(self, action_url):
    driver = self.Driver;
    if( self.IsTest):
      driver.get_screenshot_as_file( "j.png")
    else:
      driver.get(action_url)
    sleepTime = self.Randomize()
    time.sleep( sleepTime);  
    self.MoveFiles();

In [0]:
def MoveFilestoFolder(source_files, target_folder, isRemoveSource):

  #source_files= '/content/*.zip'
  #target_folder= "/content/gdrive/My Drive/Data/Data5"

  # retrieve file list
  filelist=glob.glob(source_files)
  for single_file in filelist:
      # move file with full paths as shutil.move() parameters
      #shutil.move(single_file,target_folder) 
      shutil.copy(single_file, target_folder)
      if( isRemoveSource):
        os.remove(single_file)

In [0]:
def DownloadHistoricalData( codes,  startrange, endrange, sleepTime , isTest=False):
  # Download historical data
  count = 1;
  for code in codes[startrange:endrange]:
    action_url = historicalPrice_url.replace("XXXX", code)
    print( count, action_url )
    
    # For testing purpose only capture screenshot
    if( isTest):
      driver.get_screenshot_as_file('zz.png')
    else:
      driver.get(action_url)
    time.sleep( sleepTime );
    count = count + 1;

In [0]:
def DownloadTransactionalData( codes,  dates, coderange, daterange, sleepTime, source_files, target_folder, isTest=False):
#source_files= '/content/*.zip'
#target_folder= "/content/gdrive/My Drive/NewData/"
  count = 1;
  for code in codes[0:coderange]:
    timecount = 1;
    # Create directory to store downloaded file
    filepath = target_folder + code + "/"
    if( not path.exists(filepath) ):
      os.mkdir( filepath )

    print( count, filepath )
    count = count + 1;

    for tradedate in dates[0:daterange]:
      action_url = timeSales_url.replace("XXXX", code)
      action_url = action_url.replace("yyyyyyyy", tradedate.value );
      print(timecount, action_url)
      if( isTest):
        driver.get_screenshot_as_file( code + "_" + tradedate.value + ".png")
      else:
        driver.get(action_url)
      time.sleep(sleepTime)
      timecount = timecount + 1;
    
    filelist=glob.glob(source_files)
    print(len(filelist))
    for single_file in filelist:
      # move file with full paths as shutil.move() parameters
      print( single_file, filepath)
      shutil.copy(single_file, filepath)
      os.remove(single_file)
    

In [0]:
def SendEmail(errormsg):
  import smtplib, ssl
  import time;
  from datetime import datetime
  # Import the email modules we'll need
  from email.message import EmailMessage

  now = datetime.now()
  port = 465  # For SSL
  smtp_server = "smtp.gmail.com"
  sender_email = ""  # Enter your address
  receiver_email = ""  # Enter receiver address
  password = ""
 
  message = """\
  Execution stopped at """ + str(now) + " due to " + errormsg + '.';

  msg = EmailMessage();
  msg['Subject'] = "Data extraction status"
  msg['From'] = sender_email;
  msg['To'] = receiver_email
  msg.set_content( message )

  context = ssl.create_default_context()
  with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
      server.login(sender_email, password)
      #server.sendmail(sender_email, receiver_email, message)
      server.send_message( msg )
      

In [0]:
#inipath = "/content/gdrive/My Drive/NewData"
def InspectTransactionalFile(inipath, dates):
  files = []
  directories = []
  # r=root, d=directories, f = files
  for r, d, f in os.walk(inipath):
      if( r.find(".MY") > -1 ):
        directories.append(r)

  alldates = [];
  for date in dates:
    alldates.append(date)

  items = {}
  checkdates = ';'.join(alldates)
  print(len( directories) )
  for folder in directories:
    source_files= folder + '/*.zip';
    filelist=glob.glob(source_files)
    print( folder, len(filelist) );
    if(len(filelist) < len(dates)):
      initial = inipath + "/"
      downloadedFiles = list((dlfile for dlfile in filelist if dlfile.find(".zip") > - 1 ))
      downloadedDates = list((filename[ len(filename) - 12: len(filename)-4] for filename in downloadedFiles))
      requiredDates = [];
      for date in alldates:
        try:
          downloadedDates.index(date)
        except:
          requiredDates.append(date)
      name = folder[ len(initial):  len(folder)];
      items[name] = requiredDates

  return items;

In [0]:
def AdjustData():
  inipath = "/content/gdrive/My Drive/NewData"

  files = []
  directories = []
  # r=root, d=directories, f = files
  for r, d, f in os.walk(inipath):
      if( r.find(".MY") > -1 ):
        directories.append(r)

  alldates = [];
  for date in dates:
    alldates.append(date)

  #items = {}
  checkdates = ';'.join(alldates)
  for folder in directories:
    source_files= folder + '/*.zip';
    filelist=glob.glob(source_files)
    #print( folder, len(filelist) );
    if(len(filelist) > 1 ):
      firstIndex = 56
      secondIndex = filelist[0].find("_", 56);
      name = filelist[0][firstIndex:secondIndex];
      if( len(name) > 0):
        items[name] = folder
          
  for folder in directories:
    source_files= folder + '/*.zip';
    filelist=glob.glob(source_files)
    if(len(filelist) > 1):
      for dlfile in filelist:
          secondIndex = dlfile.find("_", 56);
          name = dlfile[firstIndex:secondIndex];
          try:
            rightpath = items[name];
            if(dlfile.find(rightpath) == -1):
              print(rightpath, dlfile)
          except:
            print("F")
  for k,v in items.items():
    print(k,v)

In [0]:
# Main execution block
import random
import time
import glob
import shutil
import os
import os.path
from os import path

# if (__name__ == "__main__"):
#   # Parameter
#   isMobile            = False
#   historicalPrice_url = "http://www.shareinvestor.com/prices/price_download_zip_file.zip?type=historical&counter=XXXX"
#   timeSales_url       = "http://www.shareinvestor.com/prices/price_download_zip_file.zip?type=timesales&counter=XXXX&date=yyyyyyyy"
#   quoteMovement_url   = "http://www.shareinvestor.com/prices/price_download_zip_file.zip?type=quotemovements&counter=XXXX&date=yyyyyyyy"
#   tradeSummary_url    = "http://www.shareinvestor.com/prices/price_download_zip_file.zip?type=tsm&counter=XXXX&date=yyyyyyyy"
#   intraDay_url        = "http://www.shareinvestor.com/prices/price_download_zip_file.zip?type=intraday&counter=XXXX"
#   sleepTime           = random.randint(15, 20);
#   source_files= '/content/*.zip'
#   target_folder= "/content/gdrive/My Drive/Data/Data5"
#   ismobile = False;
#   SeleniumSetup();
#   gc = GoogleAuthentication();
#   spreadsheet = GetGoogleSpreadSheet( gc );
#   codes = GetCompanyList( spreadsheet );
#   dates = GetDateList( spreadsheet );
#   webdriver = ChromeSetup( ismobile);
#   Login( webdriver )

#   isremovefile = True;
#   istest = True;
#   downloadHistoricalData = DownloadAction("HistoricalData", istest, isremovefile, historicalPrice_url, 
#                                           source_files, target_folder, codes, webdriver, sleepTime  );
#   downloadIntraDayData = DownloadAction("IntraDayData", istest, isremovefile, intraDay_url, 
#                                         source_files, target_folder, codes, webdriver, sleepTime  );
#   downloadQuoteMovement = DownloadAction("QuoteMovement", istest, isremovefile, quoteMovement_url, 
#                                         source_files, target_folder, codes, webdriver, sleepTime, dates   );


  #downloadHistoricalData.Download( webdriver )

In [0]:
#istest = False;
import glob
import shutil
import os
import os.path
from os import path
gc = GoogleAuthentication();
spreadsheet = GetGoogleSpreadSheet( gc );
allcodes = GetCompanyList( spreadsheet );
dates = GetDateList( spreadsheet );
#dv.Login( webdriver )
inipath = "/content/gdrive/My Drive/NewData"
adjlist = InspectTransactionalFile(inipath, dates)

total = 0;
for k,v in adjlist.items():
  print(k, v)
  total = total + len(v)

totalrequired = 73 * 90
print ( "Total required    ", totalrequired )
print ( "Total pending    " ,total );
print ( "Completion status ", round( ( (totalrequired - total ) / totalrequired ) * 100, 2 ) )